In [1]:
import pulp
import pandas as pd

#### Function: calcTableCombinations
This function takes a list of guests and the maximum that can sit at a table and returns a list of all possible table seating combinations.  It will print out the topN and bottomN seating arrangements for verification

In [2]:

def calcTableCombinations(guests,max_table_size,topN,bottomN):
    
    
    # allcombinations - returns an object which contains all possible combinations
    #      guests- a list of guests
    #      max_table_size - the maximum nubmer of people per table
    #      this will return all combinations of people from 1 person per table up to 4 per table
    possible_tables = [tuple(c) for c in pulp.allcombinations(guests,max_table_size)]
    
    #total number of combinations
    total_combinations = len(possible_tables)
    
    if topN !=-1 and bottomN !=-1:
        # print out some of the table combinations
        print("First {} table combinations - out of {} possible".format(topN,total_combinations))
        for i in range(0,20):
            print(possible_tables[i])

        print("\nLast {} table combinations - out of {} possible".format(bottomN,total_combinations))
        for i in range(total_combinations-15,total_combinations):
            print(possible_tables[i])
        
    return possible_tables


#### Function: happinessScores
The happinessScores() function takes the same table input as the arbitrary happiness function which is the current seating arrangement. The scores table contains the personA-personB scores.  

+ if a table only has one person, it just looks up the score for that person with themselves
+ if a table has more than one person, all possible combinations of people at the table are determined and the score for each pair is looked up
+ the total score for each table is calculated and returned by the function

In [3]:
def happinessScores(table,scores_table,printScores=True):

    # print progress
    if printScores:
        print("\nTable:{}".format(table))
    
    # initialize
    score=0
    people=len(table)
    
    # for tables with more than 1 person
    if people>1:
        
        #get all combinations of pairs of people at each table
        combos = calcTableCombinations(table,2,-1,-1)
        
        # for each pair, calculate the score
        for pair in combos:
            # all possible combos will include single people seated with themselves. ignore those
            if len(pair)>1:
                p=list(pair)
                
                # lookup the score of the pair
                s=int(scores_table[(scores_table.Person1 ==p[0]) & (scores_table.Person2 == p[-1])]['Score'])
                
                if printScores:
                    print("Score:{:5}  Pair:{}".format(s,p))

                #increment the table score
                score+=s
    #only one person at the table, look up their score            
    else: 
        score = int(scores_table[(scores_table.Person1 ==table[0]) & (scores_table.Person2 == table[-1])]['Score'])
        
    if printScores:
        print("Table Score:{}".format(score))
    return score
     

#### Function: calcOptimalSeatingPlan
This function outputs the optimized seating plan.  There are several parameters required:
+ guests - a list  of guest names
+ max_tables - the total number of tables available
+ max_guests_per_table - number of people can sit at each table
+ happiness_function - the options are 'example'- which uses the arbitrary happiness function, 'scores' which uses the happinessScores function or 'rand' which uses the happinessRand function
+ scores - this is an optional parameter. It is a table of scores of seating arrangements. It is used for the happinessScores fuction is used. Discussed in detail below
+ printTables - determines whether the tables and scores are printed as the program executes

There are several steps performed by this function:
+ creates the pulp problem
+ calculates all possible table combinations and prints out the top 10 and bottom 10
+ creates a binary variable which determines whether a table is used or not (1 or 0)
+ creates the objective function based on which happiness function is used
+ adds constraints for the maximum number of tables and ensures every person is seated at one and only 1 table
+ solves the optimization problems
+ prints out the optimal seating arrangements

In [4]:
def calcOptimalSeatingPlan(guests
                           ,table_capacity
                           ,scores                    
                           ,printTables=False):
    
    # max number of tables
    max_tables=sum([t[1] for t in table_capacity])
    
    # max number of guests per table
    max_guests_per_table =max([c[0] for c in table_capacity])
    
    
    # create the linear problem
    seating_model = pulp.LpProblem("wedding_seating",pulp.LpMaximize)
 
    
    # calculate the number of table combinations
    if printTables:
        possible_tables =calcTableCombinations(guests,max_guests_per_table,10,10)
    else:
        possible_tables =calcTableCombinations(guests,max_guests_per_table,-1,-1)
    
    # binary variable used to flag if a table is used or not (0 or 1)
    # applied to all table combinations of people (not the max_tables)
    table_used = pulp.LpVariable.dicts('table'
                                       ,possible_tables
                                       ,lowBound=0
                                       ,upBound=1
                                       ,cat = pulp.LpInteger)
    
    
    # Objective Function- maximizes the table score
    seating_model += sum([happinessScores(table,scores,printTables)* table_used[table] for table in possible_tables])

    
    #Contraints
    # the total number of tables selected must be <= the max_tables
    seating_model += sum([table_used[table] for table in possible_tables])<= max_tables,"Max number of tables"
    
   # for k,v in tables_dict.items():
   #     seating_model += sum([table_used[table] for table in possible_tables if len(table)==int(k)])<=int(v),"Max {} person tables".format(k)
   

    #make sure the table capacity list is sorted by table size
    table_capacity = sorted(table_capacity, key = lambda x: x[0],reverse=True)

        
    for t in range(len(table_capacity)):
        capacity,number = table_capacity[t]
    
        if t<len(table_capacity)-1:
            capacity_from = table_capacity[t][0]
            capacity_to = table_capacity[t+1][0]+1
        else:
            capacity_from = table_capacity[t][0]
            capacity_to = 1
        #print(t,table_capacity[t],capacity_from,capacity_to,number)
        seating_model += sum([table_used[table] for table in possible_tables 
                              if len(table)<=capacity_from
                              and len(table)>=capacity_to ])<=number,"Max {} person tables ({} to {} people)".format(capacity_from,capacity_to,capacity_from)
   


    #every guest must have a seat,and only one seat
    for guest in guests:
        seating_model+=sum([table_used[table] for table in possible_tables 
                       if guest in table])==1,"{} must have a seat".format(guest)

    # Solve the problem    
    seating_model.solve()
    
    # Output the results
    print("\nSeating Plan: Solve Status is {}".format(pulp.LpStatus[seating_model.status]))
    t=1
    for table in possible_tables:
        if table_used[table].value()==1.0:
            print("Table Number:{} \t {}".format(t,table))  
            t+=1
    return seating_model

### Calc people combinations 
calc the possible people combinations and export to csv

In [8]:

guests = "Ken Heather Doug Jen Mark Becky Andrew Ruby".split()
table_capacity = [(6,2),(4,4),(2,2)]
#table_capacity = [(10,1),(6,2),(4,4),(2,2)]
#table_capacity = [(10,1),(6,2),(4,4),(2,2)]


max_guests_per_table = max([c[0] for c in table_capacity])


# get all possible pairs of guests, convert to a dataframe and export to csv
real_guest_combos=calcTableCombinations(guests,max_guests_per_table,10,10)
#real_guest_combos=pd.DataFrame(real_guest_combos)
#real_guest_combos.to_csv(r'real_guest_combos.csv', index = False)


First 10 table combinations - out of 246 possible
('Ken',)
('Heather',)
('Doug',)
('Jen',)
('Mark',)
('Becky',)
('Andrew',)
('Ruby',)
('Ken', 'Heather')
('Ken', 'Doug')
('Ken', 'Jen')
('Ken', 'Mark')
('Ken', 'Becky')
('Ken', 'Andrew')
('Ken', 'Ruby')
('Heather', 'Doug')
('Heather', 'Jen')
('Heather', 'Mark')
('Heather', 'Becky')
('Heather', 'Andrew')

Last 10 table combinations - out of 246 possible
('Ken', 'Heather', 'Jen', 'Becky', 'Andrew', 'Ruby')
('Ken', 'Heather', 'Mark', 'Becky', 'Andrew', 'Ruby')
('Ken', 'Doug', 'Jen', 'Mark', 'Becky', 'Andrew')
('Ken', 'Doug', 'Jen', 'Mark', 'Becky', 'Ruby')
('Ken', 'Doug', 'Jen', 'Mark', 'Andrew', 'Ruby')
('Ken', 'Doug', 'Jen', 'Becky', 'Andrew', 'Ruby')
('Ken', 'Doug', 'Mark', 'Becky', 'Andrew', 'Ruby')
('Ken', 'Jen', 'Mark', 'Becky', 'Andrew', 'Ruby')
('Heather', 'Doug', 'Jen', 'Mark', 'Becky', 'Andrew')
('Heather', 'Doug', 'Jen', 'Mark', 'Becky', 'Ruby')
('Heather', 'Doug', 'Jen', 'Mark', 'Andrew', 'Ruby')
('Heather', 'Doug', 'Jen', 'Becky

#### Determine guest scores
In this simple example we have four couples and want to arrange them at two tables. The following scoring was used:
+ if two people are a couple, score +200
+ if two people are friends, score +50
+ if two people don't know each other, score -50
+ single people are given a score of -100

The ideal result should be the following:
+ Table 1: Ken & Heather (a couple) and Doug & Jen (a couple), both couples are friends
+ Table 2: Mark & Becky (a couple) and Andrew & Rub (a couple),both couples are friends

In [9]:
# import the scores
scores = pd.read_csv("SeatingScores.csv",usecols = ['Person1','Person2','Score'])

In [13]:
table_capacity = [(4,1),(3,2)]
model=calcOptimalSeatingPlan(guests,table_capacity,scores,printTables=False)


Seating Plan: Solve Status is Optimal
Table Number:1 	 ('Ken', 'Heather')
Table Number:2 	 ('Doug', 'Jen')
Table Number:3 	 ('Mark', 'Becky', 'Andrew', 'Ruby')


### Real Test

In [14]:
lunch=pd.read_csv("lunch.csv")
lunch=lunch['Guest'].values.tolist()

# get all possible pairs of guests, convert to a dataframe and export to csv
#lunch_combos=calcTableCombinations(lunch,2,-1,-1)
#lunch_combos=pd.DataFrame(lunch_combos)
#lunch_combos.to_csv(r'lunch_combos.csv', index = False)

scores = pd.read_csv("lunch_scores.csv")

In [18]:
table_capacity = [(4,6),(2,5)]
model=calcOptimalSeatingPlan(lunch,table_capacity,scores,printTables=False)


Seating Plan: Solve Status is Optimal
Table Number:1 	 ('Tony', 'Lia')
Table Number:2 	 ('Aunt Donna', 'Evelyn')
Table Number:3 	 ('Mark', 'Becky')
Table Number:4 	 ('Heather S', 'Deric')
Table Number:5 	 ('Ken', 'Heather', 'Mary', 'Terry')
Table Number:6 	 ('Lora', 'Ralph', 'Lorielle', 'Uncle Jim')
Table Number:7 	 ('Andrew', 'Ruby', 'David', 'Erin')
Table Number:8 	 ('Uncle Danny', 'Aunt Winnie', 'Uncle Edgar', 'Aunt Beth')
Table Number:9 	 ('Chris', 'Michelle', 'Christine', 'Steve')
Table Number:10 	 ('Dan', 'Dave', 'Doug', 'Jen')
